## Selección de dataset

In [ ]:
import pandas as pd  # Librería para manipulación y análisis de datos

# Cargar los datasets
df_only_trivial = pd.read_csv('datasets/OnlyTrivial_dt.csv') # Cargar el dataset de casos triviales
df_only_non_trivial = pd.read_csv('datasets/OnlyNonTrivial_dt.csv') # Cargar el dataset de casos no triviales

# Seleccionar el dataset y crear una copia explícita
dataset = df_only_trivial.copy() # Seleccionar el dataset de casos triviales y crear una copia para manipularlo

## Información de dataset

In [ ]:
# Mostrar las primeras filas del dataset
dataset.head()

In [ ]:
# Mostrar la información del dataset
dataset.info()

In [ ]:
# Descripción estadística del dataset
dataset.describe()

In [ ]:
# Histograma de las características principales
principal_metrics = ['cbo', 'cboModified', 'fanin', 'fanout', 'wmc', 'dit', 'noc', 'rfc', 'lcom', 'lcom*']
# Generar histograma
dataset[principal_metrics].hist(bins=50, figsize=(20, 15))

## Conjunto de pruebas

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir el dataset en un conjunto de entrenamiento y uno de prueba
train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)

# Mostrar la cantidad de datos en cada conjunto
print("Largo de conjuntos: ")
print("Entrenamiento: ", len(train_set))
print("Prueba: ", len(test_set))

In [ ]:
# Estabilización de los conjuntos de entrenamiento y prueba
import numpy as np
from zlib import crc32

def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

# Como el dataset no tiene una columna de identificación, se creará 
# una columna con el índice de cada fila
dataset_with_id = dataset.reset_index()
# Dividir el dataset en entrenamiento y prueba
train_set, test_set = split_train_test_by_id(dataset_with_id, 0.2, "index")

## Exploración

In [ ]:
# Copia de dataset para manipulación
exploration_dataset = train_set.copy()

In [ ]:
# Escalado de las características
import seaborn as sns  # Librería para visualización de datos
import matplotlib.pyplot as plt  # Importar matplotlib para visualización
from sklearn.preprocessing import StandardScaler  # Escalador
from sklearn.pipeline import Pipeline  # Pipeline para encadenar pasos

# Seleccionar las columnas a escalar
columns_to_scale = ['cbo', 'cboModified', 'fanin', 'fanout', 'wmc', 'dit', 'noc', 'rfc', 'lcom', 'lcom*']

# Asegurar que las columnas sean de tipo float64
exploration_dataset[columns_to_scale] = exploration_dataset[columns_to_scale].astype('float64')

# Crear un pipeline para el escalado
pipeline = Pipeline([
    ('scaler', StandardScaler())
])

# Aplicar el escalado a las columnas seleccionadas
exploration_dataset[columns_to_scale] = pipeline.fit_transform(exploration_dataset[columns_to_scale])

# Mapa de calor de correlaciones
plt.figure(figsize=(10, 8))
sns.heatmap(dataset[columns_to_scale].corr(), annot=True, cmap='coolwarm')
plt.show()

## Preparación de datos

In [ ]:
# Limpieza de datos nulos mediante reemplazo por la mediana con SimpleImputer

from sklearn.impute import SimpleImputer  # Importar SimpleImputer

# Crear un SimpleImputer con la estrategia de reemplazo por la mediana
# Se ocupa la mediana por ser más robusta a valores atípicos
imputer = SimpleImputer(strategy='median') 

# Seleccionar las columnas numéricas
numerical_columns = dataset.select_dtypes(include=[np.number]).columns.tolist()

# Aplicar el imputer a las columnas numéricas
imputer.fit(dataset[numerical_columns])

# Transformar el dataset
dataset[numerical_columns] = imputer.transform(dataset[numerical_columns])

# Verificar si hay valores nulos
dataset.isnull().sum()

In [ ]:
# Paso de variables categóricas a numéricas Parte 1

# Importar OrdinalEncoder
from sklearn.preprocessing import OrdinalEncoder


# Seleccionar las columnas categóricas (Solo se debe ocupar la columna 'type')
categorical_columns = ['type']

# Crear un OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

# Aplicar el OrdinalEncoder a las columnas categóricas
type_encoded = ordinal_encoder.fit_transform(dataset[categorical_columns])

# Mostrar las categorías
ordinal_encoder.categories_

In [ ]:
# Paso de variables categóricas a numéricas Parte 2

# Importar OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

# Crear un OneHotEncoder
cat_encoder = OneHotEncoder()

# Aplicar el OneHotEncoder a las columnas categóricas
type_1hot = cat_encoder.fit_transform(dataset[categorical_columns])

# Mostrar las categorías
type_1hot.toarray()

## Transformación de datos

In [ ]:
# Crear un pipeline para la transformación de datos

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Definir pipeline para las columnas numéricas
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

# Seleccionar las columnas numéricas (metricas principales)
numerical_columns = ['cbo', 'cboModified', 'fanin', 'fanout', 'wmc', 'dit', 'noc', 'rfc', 'lcom', 'lcom*']

# Aplicar el pipeline a las columnas numéricas
dataset_transformed = num_pipeline.fit_transform(dataset[numerical_columns])

# Limpiar valores nulos
#dataset_transformed = dataset.dropna(subset=numerical_columns)

# Mostrar el resultado
dataset_transformed

In [ ]:
# Combinar transformaciones

from sklearn.compose import ColumnTransformer

# Seleccionar las columnas categóricas
categorical_columns = ['type']

# Definir las columnas numéricas y categóricas
numerical_columns = ['cbo', 'cboModified', 'fanin', 'fanout', 'wmc', 'dit', 'noc', 'rfc', 'lcom', 'lcom*']

# Crear un ColumnTransformer
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, numerical_columns),
    ('cat', OneHotEncoder(), categorical_columns)
])

# Aplicar el ColumnTransformer al dataset
dataset_prepared = full_pipeline.fit_transform(dataset)

# Mostrar el resultado
dataset_prepared

## Entrenamiento y evaluación de modelos en el conjunto de entrenamiento

### Regresión Lineal

In [ ]:
# Entrenamiento y evaluación de modelos en el conjunto de entrenamiento

from sklearn.linear_model import LinearRegression  # Importar modelo de regresión lineal
from sklearn.metrics import mean_squared_error # Importar métrica de error cuadrático medio

# Seleccionar el modelo
lin_reg = LinearRegression()

# Entrenar el modelo
lin_reg.fit(dataset_prepared, dataset['refactoring'])

# Predecir en todo el conjunto de entrenamiento
dataset_predictions = lin_reg.predict(dataset_prepared)

# Calcular el error
lin_mse = mean_squared_error(dataset['refactoring'], dataset_predictions)
lin_reg_mse = np.sqrt(lin_mse)
print("Error cuadrático medio: ", lin_reg_mse)
# Raíz del error cuadrático medio
rmse = np.sqrt(lin_reg_mse)
print("Raíz del error cuadrático medio: ", rmse)

### Decision Tree Regressor

In [ ]:
# Entrenamiento y evaluación de modelos en el conjunto de entrenamiento

from sklearn.tree import DecisionTreeRegressor  # Importar modelo de árbol de decisión

# Seleccionar el modelo
tree_reg = DecisionTreeRegressor()

# Entrenar el modelo
tree_reg.fit(dataset_prepared, dataset['refactoring'])

# Predecir en todo el conjunto de entrenamiento
dataset_predictions = tree_reg.predict(dataset_prepared)

# Calcular el error
tree_mse = mean_squared_error(dataset['refactoring'], dataset_predictions)

# Calcular la raíz del error cuadrático medio
tree_reg_mse = np.sqrt(tree_mse)
print("Error cuadrático medio: ", tree_reg_mse)
# Raíz del error cuadrático medio
rmse = np.sqrt(tree_reg_mse)
print("Raíz del error cuadrático medio: ", rmse)

### Random Forest Regressor

In [ ]:
# Entrenamiento y evaluación de modelos en el conjunto de entrenamiento

from sklearn.ensemble import RandomForestRegressor  # Importar modelo de bosque aleatorio

# Seleccionar el modelo
forest_reg = RandomForestRegressor()

# Entrenar el modelo
forest_reg.fit(dataset_prepared, dataset['refactoring'])

# Predecir en todo el conjunto de entrenamiento
dataset_predictions = forest_reg.predict(dataset_prepared)

# Calcular el error
forest_mse = mean_squared_error(dataset['refactoring'], dataset_predictions)

# Calcular la raíz del error cuadrático medio
forest_reg_mse = np.sqrt(forest_mse)
print("Error cuadrático medio: ", forest_reg_mse)
# Raíz del error cuadrático medio
rmse = np.sqrt(forest_reg_mse)
print("Raíz del error cuadrático medio: ", rmse)

## Validación cruzada

In [ ]:
# Validación cruzada

from sklearn.model_selection import cross_val_score  # Importar validación cruzada

# Función para mostrar los resultados
def display_scores(scores):
    print("Scores: ", scores)
    print("Mean: ", scores.mean())
    print("Standard deviation: ", scores.std())

In [ ]:
# Validación cruzada para el modelo de regresión lineal
lin_scores = cross_val_score(lin_reg, dataset_prepared, dataset['refactoring'], scoring='neg_mean_squared_error', cv=10)
lin_reg_mse_scores = np.sqrt(-lin_scores)

# Mostrar los resultados
print("Regresión lineal:")
display_scores(lin_reg_mse_scores)

In [ ]:
# Validación cruzada para el modelo de árbol de decisión
tree_scores = cross_val_score(tree_reg, dataset_prepared, dataset['refactoring'], scoring='neg_mean_squared_error', cv=10)
tree_reg_mse_scores = np.sqrt(-tree_scores)

# Mostrar los resultados
print("Árbol de decisión:")
display_scores(tree_reg_mse_scores)

In [ ]:
# Validación cruzada para el modelo de bosque aleatorio
forest_scores = cross_val_score(forest_reg, dataset_prepared, dataset['refactoring'], scoring='neg_mean_squared_error', cv=10)
forest_reg_mse_scores = np.sqrt(-forest_scores)

# Mostrar los resultados
print("Bosque aleatorio:")
display_scores(forest_reg_mse_scores)